In [1]:
import pandas as pd
import numpy as np
import scipy as sp
pd.options.display.max_rows = 400
pd.options.display.max_columns = 400
%load_ext autoreload
%autoreload 2

In [128]:
from loadcreon import LoadCreon
from creonmetrics import pu_scorer, prior_squared_error_scorer_015, \
    brier_score_labeled_loss_scorer, f1_assumed_scorer, f1_labeled_scorer
from semisuperhelper import SemiSupervisedHelper

In [3]:
path = "C:\Data\\010317\membership14_final_0103.txt"
lc = LoadCreon(path)

## Let's stratify 80/20 train test holdout for future use

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(lc.X, lc.y, test_size=0.2, random_state=771, stratify=lc.y)

## Let's just train on PN data for now

In [24]:
semi_helper = SemiSupervisedHelper(y_train.values)

In [25]:
X_train_pn, y_train_pn = semi_helper.pn(X_train.values)

In [26]:
X_t, y_t = semi_helper.pn_assume(X_train.values, unlabeled_pct=345353453)

In [27]:
X_t.shape, y_t.shape

((352556, 288), (352556,))

In [130]:
y_t[:20], y_t[-20:]

(array([1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0], dtype=int64),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64))

## Quick Scaler and LASSO to test run-time

In [129]:
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from pnuwrapper import PNUWrapper

In [130]:
estimators = [('scaler', MaxAbsScaler()),
              ('clf',PNUWrapper(base_estimator=LogisticRegression(penalty='l1', C=10), 
                                num_unlabeled=5819, threshold_set_pct=0.0143))]
pipe = Pipeline(estimators)

In [131]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from creonmetrics import pu_scorer

In [132]:
scores = cross_val_score(pipe, X_train, y_train, cv=2, scoring=f1_labeled_scorer, n_jobs=4)

JoblibAssertionError: JoblibAssertionError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\jeffrey.gomberg\AppData\Local\Continuum\Anaconda3\envs\creon\lib\runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    179         sys.exit(msg)
    180     main_globals = sys.modules["__main__"].__dict__
    181     if alter_argv:
    182         sys.argv[0] = mod_spec.origin
    183     return _run_code(code, main_globals, None,
--> 184                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...eon\\lib\\site-packages\\ipykernel\\__main__.py')
    185 
    186 def run_module(mod_name, init_globals=None,
    187                run_name=None, alter_sys=False):
    188     """Execute a module's code without importing it

............................................................

In [ ]:
scores

In [ ]:
scores.mean(), scores.std()*2

In [83]:
pipe.get_params()

{'clf': PNUWrapper(base_estimator=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
       num_unlabeled=5819, random_state=None, threshold_set_pct=0.0143),
 'clf__base_estimator': LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
 'clf__base_estimator__C': 10,
 'clf__base_estimator__class_weight': None,
 'clf__base_estimator__dual': False,
 'clf__base_estimator__fit_intercept': True,
 'clf__base_estimator__intercept_scaling': 1,
 'clf__base_estimator__max_iter': 100,
 'clf__base_estimator__multi_class': 'ovr',
 'clf__base_estimator__n_jobs': 1,
 'clf__base_